<a href="https://colab.research.google.com/github/meti-94/TextClassification/blob/main/Embeddings_Finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Download full corpus from LINDAT repository
!curl --remote-name-all https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-3195{/README.md,/lscp-0.5-fa.7z,/lscp-0.5-fa-normalized.7z,/lscp-0.5-fa-derivation-tree.7z,/lscp-0.5-fa-cs.7z,/lscp-0.5-fa-en.7z,/lscp-0.5-fa-de.7z,/lscp-0.5-fa-it.7z,/lscp-0.5-fa-hi.7z}

In [ ]:
#Decompress all files
!7z -y e "lscp-0.5-*.7z"

In [ ]:
# Install main transformers library
!pip install transformers==4.1.1

     |████████████████████████████████| 1.5MB 8.1MB/s 
     |████████████████████████████████| 890kB 29.9MB/s 
     |████████████████████████████████| 2.9MB 44.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=22019443ec42ae3088c140fa47d8a199377ea515d5fac4ef8329657291427428
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
# Import necessary libraries 
import re
from transformers import AutoConfig, AutoTokenizer, AutoModel, BertForMaskedLM
from transformers import LineByLineTextDataset
from transformers import DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer

In [ ]:
import re
# remove all non persian chars from text
def eliminate_non_persian_chars(text):
    space_codepoints ='\u0020\u2000-\u200F\u2028-\u202F'
    persian_alpha_codepoints = '\u0621-\u0628\u062A-\u063A\u0641-\u0642\u0644-\u0648\u064E-\u0651\u0655\u067E\u0686\u0698\u06A9\u06AF\u06BE\u06CC'
    punctuation_marks_codepoints = '\u060C\u061B\u061F\u0640\u066A\u066B\u066C'
    regex = re.compile(f'[^{persian_alpha_codepoints+space_codepoints+punctuation_marks_codepoints}]')
    persian_text = regex.sub('', text)
    regex = re.compile('\s+')
    output_text = regex.sub(' ', persian_text)
    return output_text

# split giant raw file into more managable files (due to memory errors!)
lines_per_file = 500000
counter = 0 
file_counter = 1
content = []
with open('lscp-0.5-fa.txt', 'r', encoding='utf-8') as fin:
    for line in fin:
        counter += 1
        line = eliminate_non_persian_chars(line)
        content.append(line)
        if counter >= lines_per_file:
            with open(f'train_data/file_no.{file_counter}.txt', 'w', encoding='utf-8') as fout:
                fout.write('\n'.join(content))
            file_counter+=1
            content = []
            counter = 0
    


FileNotFoundError: ignored

In [ ]:
# load up specific genre corpus, in this case comments feedback 
import pandas as pd 
from sklearn.model_selection import train_test_split
df = pd.read_excel('drive/My Drive/train.xlsx').fillna('')
# print(df.head()) 
X = df['text'].to_list(); y = df['label'].to_list()
df = pd.read_excel('drive/My Drive/taghche.xlsx').fillna('')
# print(df[['comment', 'label_id']].head()) 
X2 = df['comment'].to_list(); y2 = df['label_id'].to_list()
df = pd.read_excel('drive/MyDrive/snappfood.xlsx')
# print(df[['comment', 'label_id']].head()) 
X3 = df['comment'].to_list(); y3 = df['label_id'].apply(lambda x:1-x).to_list()
X+=X2; y+=y2
X+=X3; y+=y3
train_texts, valid_texts, train_labels, valid_labels = train_test_split(X, 
                                                                       y, 
                                                                       random_state=42, 
                                                                       stratify=y, 
                                                                       test_size=.1)

# save as train and validation data
with open('train.txt', 'w', encoding='utf-8') as fout:
  fout.write('\n'.join(train_texts))
with open('valid.txt', 'w', encoding='utf-8') as fout:
  fout.write('\n'.join(valid_texts))


In [ ]:
# load up preferred transformers model
config = AutoConfig.from_pretrained("HooshvareLab/bert-fa-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-fa-base-uncased")
model = BertForMaskedLM.from_pretrained("HooshvareLab/bert-fa-base-uncased")

Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# convert raw text file to proper dataset object (based on task)
from transformers import LineByLineTextDataset
print('step!')
dataset_1 = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="train.txt",
    block_size=128,
)

# print('step!')
dataset_2 = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="valid.txt",
    block_size=128,
)


/usr/local/lib/python3.6/dist-packages/transformers/data/datasets/language_modeling.py:128: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


step!


In [ ]:
# create dataloader object 
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer = tokenizer, mlm = True, mlm_probability = 0.15
)

In [ ]:
# transformers API for train :)
training_args = TrainingArguments(
    output_dir="./sentiment_small",
    overwrite_output_dir=True,
    num_train_epochs=20,
    per_gpu_train_batch_size=32,
    save_steps=10_000,
    save_total_limit=2,
    do_eval=True,
    evaluation_strategy = 'epoch'
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset_1, 
    eval_dataset = dataset_2,
    # prediction_loss_only=True,
)

In [ ]:
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Epoch,Training Loss,Validation Loss


In [ ]:
# save model 
trainer.save('./models/fine_tuned_parsbert')